In [ ]:
import time
import random

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import load_model

In [ ]:
from imagegenerator import HDR2SDRImageGenerator
from singlecsvreader import SingleCSVReader
from lutmaker import predict, write_lut_fast

In [ ]:
tf.config.list_physical_devices('GPU')

## Create the data generator

In [ ]:
csv_types = {
    'hr': np.uint16,
    'hg': np.uint16,
    'hb': np.uint16,
    'sr': np.uint8,
    'sg': np.uint8,
    'sb': np.uint8,
}

csv_file = "../data/s07e01_1.csv"


image_gen = SingleCSVReader(
    csv_file=csv_file,
    batch_size=2048,
    csv_dtypes=csv_types,
    x_cols=['hr', 'hg', 'hb'],
    y_cols=['sr', 'sg', 'sb']
)

image_gen.load()

## Train the model

In [ ]:
def timestamp():
    return str(int(time.time()))


def build_model():
    model = keras.Sequential([
        layers.Dense(8, activation=tf.nn.relu, input_shape=[3]),
        layers.Dense(8, activation=tf.nn.relu),
        layers.Dense(3)
    ])
    
    optimizer = tf.keras.optimizers.Nadam(learning_rate=0.001)
    
    # loss: mean_squared_error or mean_absolute_error
    model.compile(loss='mean_absolute_error',
                  optimizer=optimizer,
                  metrics=['mean_absolute_error', 'mean_squared_error'])
    return model

In [ ]:
model = build_model()
display(model.summary())

In [ ]:
cp_filepath = "../checkpoints/weights-" + timestamp() + "-{loss:.2f}.hdf5"
checkpoint = keras.callbacks.ModelCheckpoint(cp_filepath, monitor='loss')

model.fit(image_gen, epochs=3, callbacks=[checkpoint], shuffle=True)

## Test and output LUT

In [ ]:
predict(model, 65535, 61937, 771) # The yellow of the CW logo - should be (133, 132, 81)

In [ ]:
write_lut_fast("../tf_lut.cube", model, 65)